## Define classes to organize network creation logic
One collaboration graph is built for the whole 90 day window, so this may need to be refactored to do the coefficient calculations per team per day.

In [7]:
import random
from datetime import datetime, date, timedelta
from itertools import chain
import pandas as pd
import networkx as nx
from itertools import combinations  
import numpy as np
import time

class CollaborationNetworkGraph:
    def __init__(self, note_df, notes_for_care_date):
        """notes_df is assumed to be sorted by date"""
        self.note_df = note_df
        self.notes_for_care_date = notes_for_care_date
        self.G = nx.Graph() # this will not be used
        self.unique_dates = note_df.date.unique()
        self.__create_graph()
        
    def __create_graph(self):
        discharge_ids_for_care_date = self.notes_for_care_date.discharge_id.unique()
        care_list = []
        for discharge_id in discharge_ids_for_care_date:
                drs_for_discharge_id = self.notes_for_care_date.query('discharge_id == @discharge_id').dr.unique()
                df = pd.DataFrame(list(combinations(drs_for_discharge_id, 2)), columns=['DOC_1', 'DOC_2'])
                df["Edge"] = df["DOC_1"] + df["DOC_2"]
                df["discharge_id"] = discharge_id
                care_list.append(df)
        print(care_list.info())
        
        discharge_ids_for_experience_window = self.note_df.discharge_id.unique()
        experience_list = []
        for discharge_id in self.note_df:
                drs_for_discharge_id = self.note_df.query('discharge_id == @discharge_id').dr.unique()
                df = pd.DataFrame(list(combinations(drs_for_discharge_id, 2)), columns=['DOC_1', 'DOC_2'])
                df["Edge"] = df["DOC_1"] + df["DOC_2"]
                df["discharge_id"] = discharge_id
                experience_list.append(df)
        print(experience_list)
        
        care_edge_list = care_list["Edge"].tolist()  ###Create list of edges for FirstRow
        experience_team_edge_rows = experience_list["Edge"].isin(care_edge_list)  ##Subsetting experience row boolean from the sample period
        filtered_experience_list = experience_list[experience_team_edge_rows]
        print(filtered_experience_list)
        
        
#         for note_date in self.unique_dates[:-1]:
#             notes_for_date = self.note_df.query('date == @note_date')
#             discharge_ids_for_date = notes_for_date.discharge_id.unique()
#             for discharge_id in discharge_ids_for_date:
#                 drs_for_discharge_id = notes_for_date.query('discharge_id == @discharge_id').dr.unique()
#                 for comb in list(combinations(drs_for_discharge_id, 2)): self.__add_edge_to_G(*comb)
                # this is where the calculations may need to go...
                # this method could be public and yield results for each team
        
    def __add_edge_to_G(self, name1, name2):
        edge = (name1, name2)
        data = self.G.get_edge_data(*edge, default=None)
        weight = 1 if data is None else data['weight'] + 1
        self.G.add_edge(*edge, weight=weight)
        
class GraphGenerator:
    def __init__(self, note_df, window_in_days, step_in_days):
        note_df.sort_values('date', inplace=True)
        self.note_df = note_df
        self.DELTA = np.timedelta64(window_in_days, 'D')
        step = np.timedelta64(step_in_days, 'D')
        first_date = note_df['date'].iloc[0]
        last_date = note_df['date'].iloc[-1]
        self.date_range = np.arange(first_date, last_date - self.DELTA, step)
        
    def __iter__(self):
        for start_date in self.date_range:
            end_date = start_date + self.DELTA
            care_date = end_date + np.timedelta64(1, 'D')
            notes_in_window = self.note_df.query('date >= @start_date & date <= @end_date')
            notes_for_care_date = self.note_df.query('date == @care_date')
            graph = CollaborationNetworkGraph(notes_in_window, notes_for_care_date)
            yield graph

In [8]:
note_df = pd.read_csv('./data/notes.csv', parse_dates=[2])

WINDOW = 90
STEP = 1

get_graphs = GraphGenerator(note_df, WINDOW, STEP)
    
for graph in get_graphs:
    pass
#     clust_coefficients = nx.clustering(graph.G)
#     print(clust_coefficients)
    

AttributeError: 'list' object has no attribute 'info'

In [6]:
l = [1,2,3]
print(l[:-1])

[1, 2]
